In [1]:
# import pytorch libraries
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from torch_geometric.deprecation import deprecated

import os
import csv
import copy
import math
import time
import random
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse
from decimal import Decimal
from scipy import stats
from scipy.stats import spearmanr, rankdata, norm
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [2]:
# Load normalized gtex data
data_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/12052023/'
gt = pd.read_csv(data_dir+"new_normed_gtex_gtex_allen_gene.txt", low_memory=False, index_col=0, sep="\t")

In [3]:
gt

,GTEX-1192X-0011-R10a-SM-DO941,GTEX-1192X-0011-R5a-SM-DNZZA,GTEX-1192X-0011-R6a-SM-DNZZB,GTEX-1192X-0011-R7b-SM-DNZZC,GTEX-1192X-0011-R8b-SM-DO11U,GTEX-11DXW-0011-R1a-SM-DNZZD,GTEX-11DXW-0011-R3b-SM-DNZZE,GTEX-11DXW-0011-R5b-SM-DO11D,GTEX-11DXW-0011-R6a-SM-DO11N,GTEX-11DXW-0011-R7b-SM-DO119,...,GTEX-ZVZQ-0011-R4b-SM-57WCZ,GTEX-ZVZQ-0011-R5b-SM-57WDC,GTEX-ZVZQ-0011-R6b-SM-57WAY,GTEX-ZVZQ-0011-R7b-SM-57WBB,GTEX-ZVZQ-0011-R8a-SM-51MR5,GTEX-ZXG5-0011-R10a-SM-57WDD,GTEX-ZXG5-0011-R5b-SM-57WBN,GTEX-ZXG5-0011-R6a-SM-57WBZ,GTEX-ZXG5-0011-R7b-SM-57WCC,GTEX-ZXG5-0011-R8a-SM-57WD1
region,Frontal_Cortex_BA9,Caudate_basal_ganglia,Nucleus_accumbens_basal_ganglia,Putamen_basal_ganglia,Hypothalamus,Hippocampus,Anterior_cingulate_cortex_BA24,Caudate_basal_ganglia,Nucleus_accumbens_basal_ganglia,Putamen_basal_ganglia,...,Amygdala,Caudate_basal_ganglia,Nucleus_accumbens_basal_ganglia,Putamen_basal_ganglia,Hypothalamus,Frontal_Cortex_BA9,Caudate_basal_ganglia,Nucleus_accumbens_basal_ganglia,Putamen_basal_ganglia,Hypothalamus
subject,GTEX-1192X,GTEX-1192X,GTEX-1192X,GTEX-1192X,GTEX-1192X,GTEX-11DXW,GTEX-11DXW,GTEX-11DXW,GTEX-11DXW,GTEX-11DXW,...,GTEX-ZVZQ,GTEX-ZVZQ,GTEX-ZVZQ,GTEX-ZVZQ,GTEX-ZVZQ,GTEX-ZXG5,GTEX-ZXG5,GTEX-ZXG5,GTEX-ZXG5,GTEX-ZXG5
A1BG,-1.0609065014249923,0.1264239078327331,0.5148757621715246,0.588289446331391,-1.771613407678612,1.6978439236413676,0.3932091801194246,1.4801508340383251,-0.2147273736360117,1.5952158126759914,...,0.5163158858752855,-0.3246145193310041,-0.5091258922853752,1.176396124442838,-0.8885490700739215,-0.2614278589963243,-0.123882321258613,1.0092030513011092,0.0327887390146478,-0.4777940721655931
A2M,-0.3259438994702136,0.6634502165382117,-0.3823344838209241,0.8866794429910266,0.3918473238070559,-0.6462636078650708,-0.9363469789541222,-0.2380127215108003,-0.7592782647155094,1.0008457065936749,...,0.0390972201647005,0.1493369929874266,-0.6888055000021085,0.2289426628534391,-0.3986639351466136,-0.722773723295188,-0.2018420057931938,-0.6109276409054578,0.4013957572006627,0.4441396106266967
A2ML1,0.5322289934322247,1.9928519061306584,0.7326323477704342,1.95738848854114,-0.3193027047079684,0.3021004981385619,0.6634502165382117,0.9343941706335293,-0.3755607691893882,0.4497128493832243,...,0.8979442131538682,1.6088649489713986,1.1226894351665453,-0.1353260055415081,0.1748888113730733,0.8960588596621761,1.3161348891608369,0.4385801333361819,1.094714233035716,-0.1340536369893952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,-1.4992730266925278,-0.2994620483153727,-0.4000294730034018,0.2173086623150939,-0.591290258230665,-0.1455130221226983,-0.6033473103603407,0.8663141521808834,0.3715048161950432,0.8553559451025562,...,-0.2849872324504064,0.5512111556884931,0.5867910262845815,0.9701094246784554,0.0226977782392312,-1.2283492364529145,0.1327814854227067,0.0656127766985439,0.8810890606408079,-0.5988157254156958
ZYG11B,1.8027430907391908,-0.2863006147481918,-0.0353119608314343,-0.4806226588755429,1.5731064789417957,-0.1416911778834665,0.5748505821262617,-0.9188980626667594,0.5177570811930351,-1.4293018134072648,...,0.1966973470096366,-0.8055139860387096,-0.0138701182615907,-0.964071539514268,0.6063752591849039,0.5497440049149145,-0.8283988316306032,-0.2902437364955883,-0.9660802544456248,0.6744897501960817
ZYX,0.0454072576636208,-1.577466916200561,-1.127437990992025,-1.1942137490551366,-0.7409025733519962,-0.964071539514268,0.6018354095723709,-0.092174552477814,0.020175439267082,0.1531631488259466,...,-0.5453496267474041,-0.6665960684308483,-0.5748505821262617,-1.1763961244428374,-0.1149929573606819,0.4623042126606105,-1.1814486345934392,-0.1506121326041637,-1.0050156044940677,-0.2575157167511276
ZZEF1,-1.2364326481125962,-0.7796215252577617,-0.8755260797151984,-0.0567700523402176,-0.3539984561420949,-0.6650223198196294,-0.0934408738335693,0.3219574764987721,1.0092030513011092,0.4637077514571792,...,-0.3877660998272049,0.1315095487093334,-0.9782153889713696,0.0731963141600561,-0.3

In [4]:
# Define a function to apply inverse normal transformation on a pandas Series
def inverse_normal_transform(series):
    # Rank the data (average rank for ties)
    ranks = rankdata(series)
    # Convert ranks to uniform values between (0, 1) by dividing by length+1
    uniform = (ranks - 0.5) / len(ranks)
    # Apply the inverse normal transformation
    return pd.Series(norm.ppf(uniform))

In [5]:
# region_pick = ['Amygdala', 'Anterior_cingulate_cortex_BA24', 'Caudate_basal_ganglia', 
#                'Cerebellar_Hemisphere', 'Frontal_Cortex_BA9', 'Hippocampus', 'Hypothalamus', 
#                'Nucleus_accumbens_basal_ganglia', 'Putamen_basal_ganglia', 'Substantia_nigra']

# pred_region = region_pick[0]
# t_epoch = 300
# f_epoch = 500
# missing_N = 5
# save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/ATG_8_or_less/Prediction/'
# csv_file = f'{pred_region}_{t_epoch}_{f_epoch}_{missing_N}_LMfromGTEx.csv'
# go_mat = pd.read_csv(save_dir+csv_file, index_col=0)

# # data = go_mat.iloc[0, :].tolist()
# # plt.hist(data, bins=5, edgecolor='black')

# # Apply the function across the DataFrame row-wise
# transformed_df = go_mat.apply(inverse_normal_transform, axis=1)
# transformed_df.index = go_mat.index
# transformed_df.columns = go_mat.columns

# # data = transformed_df.iloc[0, :].tolist()
# # plt.hist(data, bins=5, edgecolor='black')

# transformed_df.reset_index(inplace=True)
# transformed_df.rename(columns={'index': 'Gene'}, inplace=True)

# # exp_mat_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/GTEx_Analysis_v8_eQTL_expression_matrices/'
# # sub_gt_filename = 'Brain_Amygdala.v8.normalized_expression.bed.gz'
# match_file = '/project/pi_rachel_melamed_uml_edu/adni/data/ensg2gene_name.txt'
# ensg2gene = pd.read_csv(match_file, sep='\t')
# all_bedgz = pd.read_table('/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/09032023/Gtex_from_raw_to_normalized/normed_all_10_brain_tissues.expression.bed.gz')
# gene_list = [ensg2gene['gene_name'][ensg2gene['Unnamed: 0'].tolist().index(all_bedgz['gene_id'][i])] for i in range(all_bedgz.shape[0])]
# gene_name = pd.Series(gene_list, name='Gene')
# mat = pd.concat([all_bedgz.iloc[:, :4], gene_name], axis=1)
# # Perform a left merge on 'RowName'
# mat_merge = pd.merge(mat, transformed_df, on='Gene', how='left')
# mat = mat_merge.dropna().reset_index(drop=True)
# # remove duplicated genes
# duplicated_row = mat.duplicated(subset='Gene', keep='first')
# final_mat = mat[~duplicated_row].reset_index(drop=True)
# # remove the 'Gene' column
# final_mat.drop('Gene', axis=1, inplace=True)

# save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/new_bedgz_file/'
# filename = f'{pred_region}_newnormed.expression.bed.gz'
# final_mat.to_csv(save_dir+filename, sep='\t', index=False, header=True, compression='gzip')

In [6]:
# concat PCs from all the tissue.txt file
data_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/GTEx_Analysis_v8_eQTL_covariates/'
data_dict = {}
files = os.listdir(data_dir)
# read all the covariate files from all the tissues
for file in files:
    if file.endswith('.v8.covariates.txt'):
        prefix = file.split('.v8.covariates.txt')[0]
        df = pd.read_csv(os.path.join(data_dir, file), low_memory=False, index_col=0, sep="\t")
        PCs = df.iloc[:5]
        others = df.iloc[-3:]
        df_sub = pd.concat([PCs, others])
        data_dict[prefix] = df_sub
# build a matrix with all the subjects
combined_df = pd.concat(data_dict.values(), axis=1)
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)

In [7]:
region_pick = ['Amygdala', 'Anterior_cingulate_cortex_BA24', 'Caudate_basal_ganglia', 
               'Cerebellar_Hemisphere', 'Frontal_Cortex_BA9', 'Hippocampus', 'Hypothalamus', 
               'Nucleus_accumbens_basal_ganglia', 'Putamen_basal_ganglia', 'Substantia_nigra']

t_epoch = 300
f_epoch = 500
missing_N = 5

for pred_region in region_pick:
    save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/ATG_8_or_less/Prediction/'
    csv_file = f'{pred_region}_{t_epoch}_{f_epoch}_{missing_N}_LMfromGTEx.csv'
    go_mat = pd.read_csv(save_dir+csv_file, index_col=0)
    
    # only use the subjects that also have genotype data
    overlapped_subjects = [x for x in combined_df.columns if x in go_mat.columns] 
    go_mat = go_mat[overlapped_subjects]
    
    # Apply the function across the DataFrame row-wise
    transformed_df = go_mat.apply(inverse_normal_transform, axis=1)
    transformed_df.index = go_mat.index
    transformed_df.columns = go_mat.columns
    # data = transformed_df.iloc[0, :].tolist()
    # plt.hist(data, bins=5, edgecolor='black')
    transformed_df.reset_index(inplace=True)
    transformed_df.rename(columns={'index': 'Gene'}, inplace=True)
    
    # exp_mat_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/GTEx_Analysis_v8_eQTL_expression_matrices/'
    # sub_gt_filename = 'Brain_Amygdala.v8.normalized_expression.bed.gz'
    match_file = '/project/pi_rachel_melamed_uml_edu/adni/data/ensg2gene_name.txt'
    ensg2gene = pd.read_csv(match_file, sep='\t')
    all_bedgz = pd.read_table('/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/09032023/Gtex_from_raw_to_normalized/normed_all_10_brain_tissues.expression.bed.gz')
    gene_list = [ensg2gene['gene_name'][ensg2gene['Unnamed: 0'].tolist().index(all_bedgz['gene_id'][i])] for i in range(all_bedgz.shape[0])]
    gene_name = pd.Series(gene_list, name='Gene')
    mat = pd.concat([all_bedgz.iloc[:, :4], gene_name], axis=1)
    # Perform a left merge on 'RowName'
    mat_merge = pd.merge(mat, transformed_df, on='Gene', how='left')
    mat = mat_merge.dropna().reset_index(drop=True)
    # remove duplicated genes
    duplicated_row = mat.duplicated(subset='Gene', keep='first')
    final_mat = mat[~duplicated_row].reset_index(drop=True)
    # remove the 'Gene' column
    final_mat.drop('Gene', axis=1, inplace=True)
    
    # need to sorted the genes and also remove genes in chrY
    # chr_order = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 
    #              'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX', 'chrY']
    chr_order = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 
                 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX']
    new_row_order = []
    chr_list = final_mat['#chr']
    for chrom in chr_order:
        for i in range(final_mat.shape[0]):
            if chr_list[i]==chrom:
                new_row_order.append(i)
    sorted_final_mat = final_mat.loc[new_row_order]
    
    # save the result
    save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/new_bedgz_file/'
    filename = f'{pred_region}_newnormed.expression.bed.gz'
    sorted_final_mat.to_csv(save_dir+filename, sep='\t', index=False, header=True, compression='gzip')

### See the file

In [9]:
import pandas as pd

region_pick = ['Amygdala', 'Anterior_cingulate_cortex_BA24', 'Caudate_basal_ganglia', 
               'Cerebellar_Hemisphere', 'Frontal_Cortex_BA9', 'Hippocampus', 'Hypothalamus', 
               'Nucleus_accumbens_basal_ganglia', 'Putamen_basal_ganglia', 'Substantia_nigra']

pred_region = region_pick[0]
save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/new_bedgz_file/'
filename = f'{pred_region}_newnormed.expression.bed.gz'
mat1 = pd.read_csv(save_dir+filename, sep='\t', compression='gzip')
# filename = f'{pred_region}_newnormed.expression.sorted.bed.gz'
# mat2 = pd.read_csv(save_dir+filename, sep='\t', compression='gzip')
mat1

,#chr,start,end,gene_id,GTEX-1192X,GTEX-11DYG,GTEX-11DZ1,GTEX-11GSP,GTEX-11NUK,GTEX-11OF3,...,GTEX-ZAB4,GTEX-ZAK1,GTEX-ZDXO,GTEX-ZE7O,GTEX-ZF28,GTEX-ZUA1,GTEX-ZV68,GTEX-ZVT3,GTEX-ZVZQ,GTEX-ZXG5
0,chr1,69090,69091,ENSG00000186092.4,-0.369009,-0.898936,-1.062786,-0.643034,0.382595,-0.423792,...,-0.861634,-1.840761,-0.120559,0.288809,0.342037,-0.396252,0.355490,-0.044324,-0.807824,0.031655
1,chr1,827521,827522,ENSG00000225880.5,-0.249427,-1.350775,0.597003,-0.107816,0.031655,-0.082381,...,0.082381,0.146105,-0.627544,0.552205,1.416717,0.382595,0.369009,0.465723,-0.508488,-0.069685
2,chr1,876753,876754,ENSG00000230368.2,-0.807824,-1.156505,0.315311,-0.006330,-1.207414,-0.031655,...,-0.107816,-0.249427,0.018991,0.508488,0.262509,0.184609,0.612202,0.494133,1.181577,0.369009
3,chr1,911434,911435,ENSG00000230699.2,-0.382595,0.918063,1.529007,-0.451658,-0.597003,0.706628,...,0.690469,-0.465723,0.998201,0.790402,1.717660,0.133321,0.288809,0.423792,0.722972,0.756257
4,chr1,923927,923928,ENSG00000187634.11,0.479879,0.581942,0.369009,0.756257,-2.428737,1.207414,...,-0.508488,1.234084,-0.031655,0.977579,0.825494,0.522948,0.567010,0.658680,0.146105,0.612202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15023,chrX,155197006,155197007,ENSG00000155959.10,0.120559,0.249427,-1.062786,-1.132140,1.234084,-1.156505,...,1.207414,-0.210428,-0.739512,-0.508488,-0.825494,0.275635,-0.018991,-0.756257,0.739512,-0.451658
15024,chrX,155264588,155264589,ENSG00000155961.4,0.807824,1.062786,0.937532,-0.315311,1.452133,0.057000,...,1.234084,-0.658680,-0.396252,-0.581942,-1.207414,-0.031655,-1.416717,-1.840761,0.627544,-0.236388
15025,chrX,155334656,155334657,ENSG00000155962.12,-0.057000,-0.107816,0.018991,-0.210428,-1.181577,-0.275635,...,-1.616156,-0.957363,-1.085325,-0.537519,1.108429,-0.158913,0.031655,0.249427,-0.522948,-0.184609
15026,chrX,155669943,155669944,ENSG00000185973.10,0.451658,0.315311,-0.315311,-0.146105,1.181577,-0.184609,...,-1.062786,0.581942,-0.197502,-1.914506,-0.937532,-0.107816,0.275635,0.044324,1.664765,-0.627544


### Generate a mini-batch bed.gz file for testing!

In [8]:
import pandas as pd

save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/GTEx_Analysis_v8_eQTL_expression_matrices/'
filename = 'Brain_Amygdala.v8.normalized_expression.bed.gz'
gtex_df = pd.read_csv(save_dir+filename, sep='\t', compression='gzip')
mini_df = gtex_df.loc[:100,]
# save the df
save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/Test_GTEx_fastQTL_results/'
mini_df_filename='Brain_Amygdala.v8.mini.normalized_expression.bed.gz'
mini_df.to_csv(save_dir+mini_df_filename, sep='\t', index=False, header=True, compression='gzip')

In [7]:
mini_df

,#chr,start,end,gene_id,GTEX-11NUK,GTEX-11ONC,GTEX-11PRG,GTEX-11TTK,GTEX-11ZU8,GTEX-11ZVC,...,GTEX-WZTO,GTEX-X4EP,GTEX-X4XX,GTEX-X585,GTEX-XLM4,GTEX-Y8DK,GTEX-YFC4,GTEX-ZAB4,GTEX-ZAK1,GTEX-ZVZQ
0,chr1,29552,29553,ENSG00000227232.5,-1.159742,-0.736316,-0.174416,0.524401,-1.238894,-0.686643,...,-0.569185,0.194028,1.281552,1.020076,0.313572,1.122762,0.480644,1.159742,-1.326678,-0.333891
1,chr1,62947,62948,ENSG00000240361.1,0.135385,-0.546656,0.459112,0.761843,1.122762,0.736316,...,-1.326678,-1.326678,-0.038573,-0.761843,-1.326678,-1.326678,-0.077204,-1.326678,-0.615141,0.057878
2,chr1,69090,69091,ENSG00000186092.4,0.233485,2.160044,-1.159742,0.253347,0.761843,0.897915,...,-0.057878,-1.608653,-0.213715,-1.608653,-0.233485,-1.608653,0.480644,-0.686643,-1.608653,-0.592007
3,chr1,135894,135895,ENSG00000268903.1,-0.213715,0.019283,-1.238894,0.957209,-1.541986,-1.374680,...,-0.354350,-0.437790,0.927155,1.683348,0.736316,0.194028,0.213715,0.135385,-0.416666,-0.592007
4,chr1,137964,137965,ENSG00000269981.1,-0.115950,-0.077204,-0.897915,0.437790,-0.395725,-1.326678,...,-0.174416,0.841621,0.354350,2.160044,0.135385,-0.502402,0.194028,-0.761843,-0.333891,-0.293381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,chr1,2015529,2015530,ENSG00000233542.1,0.988155,-0.233485,-0.019283,-1.238894,-1.869607,-1.198380,...,2.160044,-1.993984,0.662435,0.374957,0.927155,-0.416666,0.592007,-0.213715,0.077204,0.897915
97,chr1,2019297,2019298,ENSG00000187730.8,1.159742,-0.313572,1.198380,-0.869424,-1.426077,-1.869607,...,2.423196,-2.423196,-0.711260,-0.615141,0.841621,0.115950,0.814455,-0.437790,-0.662435,0.057878
98,chr1,2050469,2050470,ENSG00000067606.16,1.159742,-0.480644,0.524401,-1.481544,-1.281552,-1.869607,...,1.768825,-1.053073,-0.135385,0.135385,0.374957,-0.502402,0.736316,1.326678,-0.293381,0.077204
99,chr1,2184388,2184389,ENSG00000182873.5,-1.374680,0.135385,-1.326678,-0.354350,-1.281552,-1.122762,...,0.154871,-0.957209,2.423196,1.768825,0.869424,1.608653,0.416666,1.683348,-0.253347,-0.988155


### compare our bed.gz file with GTEx bed.gz file
#### to see how often are the same gene-snp pairs in both files

In [2]:
import pandas as pd

region_pick = ['Amygdala', 'Anterior_cingulate_cortex_BA24', 'Caudate_basal_ganglia', 
               'Cerebellar_Hemisphere', 'Frontal_Cortex_BA9', 'Hippocampus', 'Hypothalamus', 
               'Nucleus_accumbens_basal_ganglia', 'Putamen_basal_ganglia', 'Substantia_nigra']

pred_region = region_pick[0]
save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/new_bedgz_file/'
filename = f'{pred_region}_newnormed.expression.bed.gz'
our_bed = pd.read_csv(save_dir+filename, sep='\t', compression='gzip')

save_dir = '/project/pi_rachel_melamed_uml_edu/Jianfeng/Allen/src/Pytorch/02162024/eqtl_analysis/GTEx_Analysis_v8_eQTL_expression_matrices/'
filename = 'Brain_Amygdala.v8.normalized_expression.bed.gz'
GTEx_bed = pd.read_csv(save_dir+filename, sep='\t', compression='gzip')

In [3]:
our_bed

,#chr,start,end,gene_id,GTEX-1192X,GTEX-11DYG,GTEX-11DZ1,GTEX-11GSP,GTEX-11NUK,GTEX-11OF3,...,GTEX-ZAB4,GTEX-ZAK1,GTEX-ZDXO,GTEX-ZE7O,GTEX-ZF28,GTEX-ZUA1,GTEX-ZV68,GTEX-ZVT3,GTEX-ZVZQ,GTEX-ZXG5
0,chr1,69090,69091,ENSG00000186092.4,-0.369009,-0.898936,-1.062786,-0.643034,0.382595,-0.423792,...,-0.861634,-1.840761,-0.120559,0.288809,0.342037,-0.396252,0.355490,-0.044324,-0.807824,0.031655
1,chr1,827521,827522,ENSG00000225880.5,-0.249427,-1.350775,0.597003,-0.107816,0.031655,-0.082381,...,0.082381,0.146105,-0.627544,0.552205,1.416717,0.382595,0.369009,0.465723,-0.508488,-0.069685
2,chr1,876753,876754,ENSG00000230368.2,-0.807824,-1.156505,0.315311,-0.006330,-1.207414,-0.031655,...,-0.107816,-0.249427,0.018991,0.508488,0.262509,0.184609,0.612202,0.494133,1.181577,0.369009
3,chr1,911434,911435,ENSG00000230699.2,-0.382595,0.918063,1.529007,-0.451658,-0.597003,0.706628,...,0.690469,-0.465723,0.998201,0.790402,1.717660,0.133321,0.288809,0.423792,0.722972,0.756257
4,chr1,923927,923928,ENSG00000187634.11,0.479879,0.581942,0.369009,0.756257,-2.428737,1.207414,...,-0.508488,1.234084,-0.031655,0.977579,0.825494,0.522948,0.567010,0.658680,0.146105,0.612202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15023,chrX,155197006,155197007,ENSG00000155959.10,0.120559,0.249427,-1.062786,-1.132140,1.234084,-1.156505,...,1.207414,-0.210428,-0.739512,-0.508488,-0.825494,0.275635,-0.018991,-0.756257,0.739512,-0.451658
15024,chrX,155264588,155264589,ENSG00000155961.4,0.807824,1.062786,0.937532,-0.315311,1.452133,0.057000,...,1.234084,-0.658680,-0.396252,-0.581942,-1.207414,-0.031655,-1.416717,-1.840761,0.627544,-0.236388
15025,chrX,155334656,155334657,ENSG00000155962.12,-0.057000,-0.107816,0.018991,-0.210428,-1.181577,-0.275635,...,-1.616156,-0.957363,-1.085325,-0.537519,1.108429,-0.158913,0.031655,0.249427,-0.522948,-0.184609
15026,chrX,155669943,155669944,ENSG00000185973.10,0.451658,0.315311,-0.315311,-0.146105,1.181577,-0.184609,...,-1.062786,0.581942,-0.197502,-1.914506,-0.937532,-0.107816,0.275635,0.044324,1.664765,-0.627544


In [4]:
GTEx_bed

,#chr,start,end,gene_id,GTEX-11NUK,GTEX-11ONC,GTEX-11PRG,GTEX-11TTK,GTEX-11ZU8,GTEX-11ZVC,...,GTEX-WZTO,GTEX-X4EP,GTEX-X4XX,GTEX-X585,GTEX-XLM4,GTEX-Y8DK,GTEX-YFC4,GTEX-ZAB4,GTEX-ZAK1,GTEX-ZVZQ
0,chr1,29552,29553,ENSG00000227232.5,-1.159742,-0.736316,-0.174416,0.524401,-1.238894,-0.686643,...,-0.569185,0.194028,1.281552,1.020076,0.313572,1.122762,0.480644,1.159742,-1.326678,-0.333891
1,chr1,62947,62948,ENSG00000240361.1,0.135385,-0.546656,0.459112,0.761843,1.122762,0.736316,...,-1.326678,-1.326678,-0.038573,-0.761843,-1.326678,-1.326678,-0.077204,-1.326678,-0.615141,0.057878
2,chr1,69090,69091,ENSG00000186092.4,0.233485,2.160044,-1.159742,0.253347,0.761843,0.897915,...,-0.057878,-1.608653,-0.213715,-1.608653,-0.233485,-1.608653,0.480644,-0.686643,-1.608653,-0.592007
3,chr1,135894,135895,ENSG00000268903.1,-0.213715,0.019283,-1.238894,0.957209,-1.541986,-1.374680,...,-0.354350,-0.437790,0.927155,1.683348,0.736316,0.194028,0.213715,0.135385,-0.416666,-0.592007
4,chr1,137964,137965,ENSG00000269981.1,-0.115950,-0.077204,-0.897915,0.437790,-0.395725,-1.326678,...,-0.174416,0.841621,0.354350,2.160044,0.135385,-0.502402,0.194028,-0.761843,-0.333891,-0.293381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24064,chrX,155466539,155466540,ENSG00000224533.4,-0.736316,-1.218387,-0.524401,-1.218387,1.087257,1.159742,...,0.841621,0.313572,-1.218387,0.194028,-1.218387,-1.218387,0.019283,-1.218387,-1.218387,0.686643
24065,chrX,155669943,155669944,ENSG00000185973.10,1.020076,-1.198380,0.869424,2.423196,-0.019283,0.416666,...,-0.333891,0.615141,-1.768825,-1.281552,0.437790,0.115950,-0.502402,-1.159742,0.154871,1.426077
24066,chrX,155767811,155767812,ENSG00000168939.11,1.053073,-0.761843,1.238894,-1.374680,-0.174416,0.638610,...,1.541986,-0.927155,0.761843,-1.159742,-0.213715,0.897915,0.711260,-0.253347,-1.281552,1.768825
24067,chrX,155881292,155881293,ENSG00000124333.15,0.736316,-0.135385,0.233485,0.459112,1.541986,0.374957,...,0.313572,-1.541986,-0.662435,-0.019283,-0.057878,0.662435,-0.038573,0.638610,0.253347,-0.213715


In [30]:
overlapped_gene_id = sorted(list(set(our_bed['gene_id']) & set(GTEx_bed['gene_id'])))
print(f'There are {len(overlapped_gene_id)} overlapped gene_id between our bed.gz and GTEx bed.gz')
t = 0
for i in range(len(overlapped_gene_id)):
    trip1 = our_bed.iloc[our_bed['gene_id'].tolist().index(overlapped_gene_id[n]), :][:3]
    trip2 = GTEx_bed.iloc[GTEx_bed['gene_id'].tolist().index(overlapped_gene_id[n]), :][:3]
    if sum(trip1==trip2)==3:
        t = t+1
print(f'    - {t} of them have the same positions')

There are 14796 overlapped gene_id between our bed.gz and GTEx bed.gz
    - 14796 of them have the same positions
